https://github.com/cankocagil/DCGAN/blob/main/DCGAN.ipynb

In [ ]:
from __future__ import print_function
import random

import torch
import torchvision
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dataset
import torchvision.transforms as transforms
import torchvision.utils as vutils

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import argparse
import os
from tqdm import tqdm
import time
import copy
import math
from zipfile import ZipFile

from PIL import Image

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# PyTorch's versions:
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
print("NumPy Version: ",np.__version__)
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

seed = 999
print(f"Random Seed: '{seed}'")
random.seed(seed)
torch.manual_seed(seed)

PyTorch Version:  1.13.1+cu116
Torchvision Version:  0.14.1+cu116
NumPy Version:  1.22.4
Mon Feb 27 04:31:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    52W / 350W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+

In [ ]:
from pathlib import Path
USE_COLAB: bool = True
dataset_base_path = Path("/content/drive/My Drive/ECE 792 - Advance Topics in Machine Learning/Datasets/FakeFaces/DCGAN")
if USE_COLAB:
  from google.colab import drive
  
  # Mount the drive to access google shared docs
  drive.mount('/content/drive/', force_remount=True)

  if dataset_base_path.exists():
    print("Folder exists")
  else:
    print("DOESN'T EXIST. Add desired folder as a shortcut in your 'My Drive'")


Mounted at /content/drive/
DOESN'T EXIST. Add desired folder as a shortcut in your 'My Drive'


In [ ]:
model_dir = "/content/drive/My Drive/ECE 792 - Advance Topics in Machine Learning/Code/DatasetGeneration/DCGAN/models"

# We will be working with GPU:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device : ' , device)

# Number of GPUs available. 
num_GPU = torch.cuda.device_count()
print('Number of GPU : ', num_GPU)

config = { 'batch_size'        : 128,
           'image_size'        : 64,
           'n_channel'         : 3,
           'latent_space_size' : 100,
           'feature_mapG'      : 64,
           'feature_mapD'      : 64,
           'num_epochs'        : 50,
           'lr'                : 2e-4,
           'beta1'             : 0.5,
           'device'            : device,
 
}

Device :  cuda
Number of GPU :  1


In [ ]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
class Generator(nn.Module):
  def __init__(self,latent_space_size = config['latent_space_size'],
               ngf = config['feature_mapG'], n_channel = config['n_channel']):
    
    super(Generator,self).__init__()
     
    self.generator = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( latent_space_size, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, n_channel, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

  def forward(self,inp):
    return self.generator(inp)

In [ ]:
netG = Generator().to(device)

if (device.type == 'cuda') and torch.cuda.device_count() > 1:
  netG = nn.DataParallel(netG, list(range(torch.cuda.device_count())))

netG.apply(weights_init)

Generator(
  (generator): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
 

In [ ]:
import re
def get_latest_pth_model(base_path) -> Path:
  epoch_num = []
  all_files = sorted(Path(base_path).glob("*.pth"))
  for file_ in all_files:
    idx_num = re.search("--", str(file_)).span()
    idx_pt = re.search(".pt", str(file_)).span()
    epoch_num.append(int(str(file_)[idx_num[-1]:idx_pt[0]]))

  idx = epoch_num.index(np.max(epoch_num))
  return all_files[idx]

In [ ]:
custom_dcgan_model_file = get_latest_pth_model(model_dir)
print(custom_dcgan_model_file)
try :
  if custom_dcgan_model_file.exists():
    print('Wearing weights ...')
    checkpoint = torch.load(str(custom_dcgan_model_file))
    netG.load_state_dict(checkpoint['Generator_state_dict'])
    netG.to(device)

except RuntimeError as rtr:
  print('Saved Model cannot be loaded...')

/content/drive/My Drive/ECE 792 - Advance Topics in Machine Learning/Code/DatasetGeneration/DCGAN/models/Custom_DCGAN--49.pth
Wearing weights ...


In [ ]:
output_imgs_path = dataset_base_path
import shutil
if output_imgs_path.exists():
  shutil.rmtree(str(output_imgs_path))
output_imgs_path.mkdir(exist_ok=True, parents=True)
print(output_imgs_path)
print(output_imgs_path.exists())

/content/drive/My Drive/ECE 792 - Advance Topics in Machine Learning/Datasets/FakeFaces/DCGAN


In [ ]:
from torchvision.utils import save_image
from tqdm import tqdm

num_imgs_to_generate = 40000
batch_size = 1
torch.manual_seed(999)
if num_imgs_to_generate % batch_size != 0:
  raise RuntimeError(f"num_imgs_to_generate not divisible by batch_size. '{num_imgs_to_generate}' : '{batch_size}'")
iterations = int(num_imgs_to_generate / batch_size)
img_cnt = 0
for _ in tqdm(range(iterations)):
  with torch.no_grad():
    netG.eval()
    fakes = netG(torch.randn(batch_size, 100, 1, 1, dtype=torch.float, device=device))
    for fake in fakes:
      output_path = output_imgs_path / f"{img_cnt}.jpg"
      save_image(fake, output_path)
      img_cnt += 1

# img_files = sorted(output_imgs_path.glob("*.jpg"))
# print(f"Number of images generated: '{len(img_files)}'")
# output_zip_path = output_imgs_path.parent / "DCGAN.zip"
# from zipfile import ZipFile
# from tqdm import tqdm

# if output_zip_path.exists():
#   print(f"Deleting '{output_zip_path}'")
#   os.remove(str(output_zip_path))

# print(f"Writing images to zip file '{output_zip_path}'")
# with ZipFile(str(output_zip_path), mode='w') as archive:
#   for file_ in tqdm(img_files):
#     archive.write(file_)

# with ZipFile(str(output_zip_path), mode='r') as zipObj:
#   zipObj.extractall()

# list_of_images_in_archive = zipObj.namelist()
# print(f"Number of images in archive: '{len(list_of_images_in_archive)}'")
# if len(list_of_images_in_archive) != len(img_files):
#   raise RuntimeError(f"NOT ALL IMAGES WERE SUCCESSFULLY ARCHIVED")

# drive.flush_and_unmount()

100%|██████████| 40000/40000 [04:26<00:00, 150.18it/s]


In [ ]:
drive.flush_and_unmount()

KeyboardInterrupt: ignored